<h2>Uéliton Viana, Ciêntista de Dados</h2>

<a href="mailto:ueliton_viana@outlook.com"><i style="font-size:25px; margin-right:10px" class="fa fa-envelope"></i></a>
<a href="https://wa.me/5549984233608"><i style="font-size:25px; margin-right:10px" class="fa fa-whatsapp"></i></a>
<a href="https://www.linkedin.com/in/ueliton-viana/"><i style="font-size:25px; margin-right:10px" class="fa fa-linkedin"></i></a>
<a href="https://github.com/Uellwar"><i style="font-size:25px; margin-right:10px" class="fa fa-github"></i></a>
<a href="https://www.instagram.com/uelitonviana_/"><i style="font-size:25x; margin-right:10px" class="fa fa-instagram"></i></a>
<a href="https://www.youtube.com/channel/UCX-MsX0Tt-iGBScRhsDfSJA"><i style="font-size:25px; margin-right:10px" class="fa fa-youtube"></i></a>

Durante minhas férias, decidi me aventurar em um novo projeto de ciência de dados: a construção de um modelo de classificação de sentimentos em comentários de reviews de e-commerce. Utilizei [dados reais](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce) fornecidos pela [Olist](https://olist.com/pt-br/) para treinar os modelos. Para resolver o problema de classificação prévia dos sentimentos nos comentários, escolhi usar a pontuação de avaliação de compra real. Assim, os comentários com notas 1 e 2 são considerados sentimentos negativos, com nota 3 é considerado neutro e os sentimentos positivos foram atribuídos aos comentários com notas 4 e 5.

Para desenvolver este projeto, segui algumas etapas importantes: Primeiro, carreguei as bibliotecas, modelos e dados necessários. Em seguida, foi realizada uma limpeza e tratamento dos dados para que eles estivessem prontos para o treinamento do modelo. Depois, apliquei a vectorização e normalização aos dados e reduzi a dimensionalidade deles, para que os modelos pudessem processá-los de forma mais eficiente.
Utilizei validação cruzada para selecionar os melhores modelos e, em seguida, realizei o ajuste dos parâmetros dos modelos selecionados usando random search. Depois, avaliei os melhores modelos e os comparei com um modelo em ensemble. Por fim, escrevi um relatório detalhando os principais pontos do trabalho realizado e sugerindo melhorias futuras.

---

**Cronograma de etapas do projeto.**

Tópicos:

✅ 1. Coleta de dados: Coletar dados de comentários e avaliações de produtos em e-commerce.

✅ 2. Pré-processamento de dados: Pré-processar os dados coletados, incluindo remoção de ruídos e normalização de texto.

✅ 3. Seleção e treinamento de modelo: Selecionar e treinar um modelo de classificação de sentimento que atenda às necessidades do projeto, usando técnicas de aprendizado de máquina para melhorar o desempenho do modelo.

✅ 4. Avaliação do desempenho do modelo: Avaliar o desempenho do modelo de classificação de sentimentos, testando-o com dados de teste e ajustando os parâmetros de hiperparâmetros, se necessário.

✅ 5. Visualização dos resultados: Exibir os resultados da classificação de sentimentos de maneira gráfica e fácil de interpretar.

✅ 6. Conclusão e recomendações: Concluir o projeto e fornecer recomendações para possíveis aplicações futuras.

✅ 7. Publicação do projeto: Publicar o projeto e os resultados obtidos de maneira acessível para outros interessados.

---

In [ ]:
# Listagem dos arquivos disponibilizados pela Olist
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

**Dicionário de Dados**

| coluna       | tabela utilizada | o que essa coluna significa |
|:--------------|:--------------------------|:----------------------------|
| mensagem do comentário | olist_reviews.csv | a mensagem escrita pelo cliente quando ele deixou a revisão |
| título do comentário | olist_reviews.csv | o título da revisão escrito pelo cliente |
| pontuação da revisão | olist_reviews.csv | a pontuação dada pelo cliente na revisão (usada para medir o sentimento)|
| preço | olist_order_items.csv | o preço do produto comprado |
| preço do frete | olist_order_items.csv | o preço do frete do produto comprado |

<br><br>

As tabelas olist_customer, olist_order_items, olist_reviews e olist_orders **estão conectadas entre si** através das chaves primárias customer_id, order_id e product_id. A tabela olist_customer fornece informações sobre os clientes, como o ID único do cliente, o código postal e a cidade. A tabela olist_order_items fornece informações sobre os itens da compra, como o ID do pedido, o ID do produto, o ID do vendedor, o preço e o preço do frete. A tabela olist_reviews fornece informações sobre os comentários dos clientes, como o ID do pedido, a pontuação da revisão, o título do comentário, a mensagem do comentário e a data de criação da revisão. Finalmente, a tabela olist_orders fornece informações sobre os pedidos, como o ID do pedido, o ID do cliente, o status do pedido, a data de compra, a data de aprovação, a data de entrega do transportador e a data de entrega estimada.

---


### Importando os pacotes e as funções

In [ ]:
# Carregamento e manipulação dos dados
import pandas as pd
import numpy as np
import scipy

import warnings
warnings.filterwarnings("ignore")

# Limpeza de dados
import re
import unidecode

# Tokenização
import nltk
nltk.download('punkt')

# Remoção de stopwords
from nltk.corpus import stopwords

# Stemming
from nltk.stem import PorterStemmer

# Vectorização
from sklearn.feature_extraction.text import CountVectorizer

# Normalização
from sklearn.preprocessing import StandardScaler

# Separação de dados
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# Construção dos modelos
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier 

#Modelos usados e descartados
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
# from sklearn.neural_network import MLPClassifier
# from sklearn.naive_bayes import GaussianNB 
# from sklearn.naive_bayes import BernoulliNB
# from sklearn.naive_bayes import MultinomialNB 
# from sklearn.naive_bayes import ComplementNB 
# from sklearn.naive_bayes import CategoricalNB 
# from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier 
# from sklearn.neighbors import KNeighborsClassifier

# Escolha dos melhores modelos e parâmetros
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel

# Avaliação dos modelos
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

# Salvando e Carregando os modelos
import pickle

# Visualização
import plotly.express as px

# Métricas de classificação
from sklearn.metrics import classification_report

In [ ]:
class ProcessadorDados:
    def __init__(self):
        pass
    # função recebe o caminho de um arquivo e retorna um dataframe do pacote pandas com os dados contidos no arquivo.
    def importar_dados(self, caminho_arquivo,index_col=None,parse_dates=None):
        df = pd.read_csv(caminho_arquivo,index_col=index_col, parse_dates=parse_dates)
        return df
    
# instanciando a classe
processador = ProcessadorDados()

In [ ]:
# Esta função tem como objetivo compactar um conjunto de dados, onde a variável alvo é definida e a chave de junção entre os dados é definida
def compactar(dfs, target, key):
    # une os dados contidos em dfs (que é uma lista), usando o método 'outer', usando a chave key para a junção
    df_join = dfs[0].merge(right=dfs[1], how='outer', on=key)
    df_join= df_join.merge(right=dfs[2], how='outer', on=key)
    # remove os valores nulos contidos no df_join
    df_join.dropna(inplace=True, axis=0)
    # filtra o dataframe com somente os pedidos entregues
    df_join = df_join.loc[df_join['order_status']=='delivered']
    # define as colunas necessárias para o df_join
    cols = ['review_comment_message', 'review_comment_title', 'review_score', 'price', 'freight_value','order_id']
    df_join = df_join[cols]
    # remove as duplicatas do df_join
    df_join = df_join.drop_duplicates(subset='order_id', keep='first')
    # o identificador do pedido de X
    X = df_join.drop(columns=['order_id'],axis=1)
    # define a variável y como sendo o valor da variável alvo
    y = df_join[target]
    return X,y

# Esta função tem como objetivo limpar e tratar os dados, usando as opções fornecidas e ao final, retornar os dados prontos para o modelo aprender com eles
def limpa_e_trata(df, opcoes):
    # define as colunas de texto que serão tratadas e vetorizadas
    colunas = ['review_comment_message', 'review_comment_title']
    # verifica quais opções de tratamento serão usadas
    for opcao, habilitada in opcoes.items():
        # Esta condição verifica se a opção está habilitada
        if habilitada:
            # Estas condições verificam qual opção está habilitada e fazem o tratamento das colunas de acordo com a opção
            if opcao == 'remove_acentuacao':
                for col in colunas: 
                    df[col] = df[col].apply(lambda x: unidecode.unidecode(x))
            elif opcao == 'texto_e_numero':
                for col in colunas:
                    df[col] = df[col].apply(lambda x: re.sub('[^a-zA-Z0-9\s]','',x))
            elif opcao == 'remove_carac_dif':
                for col in colunas:
                    df[col] = df[col].apply(lambda x: re.sub('[\n\r]','',x))
            elif opcao == 'tokens':
                for col in colunas:
                    df[col] = df[col].apply(lambda x: nltk.word_tokenize(x))
            elif opcao == 'stemming_radical':
                for col in colunas:
                    ps = PorterStemmer()
                    df[col] = df[col].apply(lambda x: [ps.stem(y) for y in x])
            elif opcao == 'stopwords':
                for col in colunas:
                    stop_words = set(stopwords.words('portuguese'))
                    df[col] = df[col].apply(lambda x: [y for y in x if y not in stop_words])
    # cria um dataframe somente com as colunas de comentários
    df_tratado = df[['review_comment_message', 'review_comment_title']]
    # cria uma instância de vetorizador
    vetorizador = CountVectorizer()
    # vetoriza os comentários da mensagem, de modo a transformar em números que representem sua frequência de ocorrência
    vetorizado1 = vetorizador.fit_transform(df_tratado['review_comment_message'].str.join(' ')).toarray()
    # transforma a matriz de vetorização em um dataframe para que seja possível usar pd.concat
    vetorizado1 = pd.DataFrame(vetorizado1)
    # vetoriza os comentários do título
    vetorizado2 = vetorizador.fit_transform(df_tratado['review_comment_title'].str.join(' ')).toarray()
    # transforma a matriz de vetorização em um dataframe para que seja possível usar pd.concat
    vetorizado2 = pd.DataFrame(vetorizado2)
    # concatena os dataframes de vetorização das mensagens e dos títulos
    df_tratado = pd.concat([vetorizado1, vetorizado2], axis=1)
    # renomeia as colunas do dataframe para evitar colunas com numeracao/nome repetida
    df_tratado.columns = range(df_tratado.shape[1])
    # cria uma instância de normalização
    scaler = StandardScaler()
    # normaliza os dados do dataframe de vetorização
    df_vetor_norma = scaler.fit_transform(df_tratado)
    # cria um dataframe com os dados normalizados do vetorizador para que seja possível usar pd.concat
    df_coments = pd.DataFrame(df_vetor_norma)
    # cria uma instância de normalização
    scaler = StandardScaler()
    # normaliza os dados de preço e frete
    df_normalizado = scaler.fit_transform(df[['price','freight_value']])
    # cria um dataframe com os dados normalizados para que seja possível usar pd.concat
    df_items = pd.DataFrame(df_normalizado)
    # concatena os dataframes de itens e de comentários
    df_final = pd.concat([df_items, df_coments], axis=1)
    # renomeia as colunas do dataframe
    df_final.columns = range(df_final.shape[1])
    # retorna os dataframes tratados
    return df_final, df_items, df_coments


def melhores_modelos(X_train, y_train, modelos):
    # cria lista de dicionários que armazenarão os resultados
    resultados = {'modelo':[], 'acuracia':[]}
    # uma iteração para cada modelo na lista de modelos
    for modelo in modelos:
        # cria o objeto
        modelo = modelo()
        # cria o objeto de cross validation
        cv = KFold(n_splits=3, shuffle=True, random_state=42)
        # treina o modelo
        scores = cross_val_score(modelo, X_train, y_train, cv=cv, scoring='accuracy')
        for score in scores:
            # armazena os resultados
            resultados['modelo'].append(str(modelo).split('(')[0])
            resultados['acuracia'].append(score)
    # cria o dataframe de resultados
    df_metricas = pd.DataFrame(resultados)
    # gera o gráfico
    fig_boxplot = px.box(df_metricas, y='acuracia', x='modelo', color='modelo')
    # calcula a média da ponturação de cada modelo
    df_metricas = df_metricas.groupby('modelo').agg('mean').reset_index()
    # atribui uma coluna com o total de características presente nos dados
    df_metricas['n_colunas'] =  X_train.shape[1]
    # retorna o dataframe e o gráfico
    return df_metricas, fig_boxplot

# essa função reduz a dimensioalidade dos dados com base na importancia das características
def reducao_dimensionalidade(X_train, y_train, n_features):
    # Treina uma árvore de decisão
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    # Seleciona as características mais importantes com base na importância dada pelo modelo
    selector = SelectFromModel(model, max_features=n_features)
    selector.fit(X_train, y_train)
    selecionadas = selector.get_support()
    # Cria um dataframe com as características selecionadas
    X_train_reduzido = X_train.loc[:, selecionadas]
    return X_train_reduzido

# essa função recebe um modelo, os dados e uma lista de possíveis parametros e retorna o modelo otimizado e outras características extraídas desse modelo
def tunagem_de_modelo(modelo, X_train, y_train, lista_parametros):
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3)
    opt = RandomizedSearchCV(modelo, lista_parametros,cv=3,verbose=2)
    opt.fit(X_train, y_train)
    modelo_otimizado = opt.best_estimator_
    best_params = opt.best_params_
    accuracy = opt.best_score_
        
    y_pred = modelo_otimizado.predict(X_valid)
    df_summary = pd.DataFrame(classification_report(y_valid, y_pred, output_dict=True))
    conf_matrix = confusion_matrix(y_valid, y_pred)
    df_conf_matrix = pd.DataFrame(conf_matrix, index=['Real 1', 'Real 2', 'Real 3'], columns=['Predito 1', 'Predito 2', 'Predito 3'])
    return modelo_otimizado, best_params, accuracy, df_summary,df_conf_matrix

# essa função cria um modelo ensemble a partir de outros modelos e fita o ensemble em dados ainda nao vistos por nenhum modelo
def ensemble_modelos(modelos, X_train, y_train):
    from sklearn.ensemble import VotingClassifier
    model_list = []
    for model in modelos:
        model_list.append((str(model), model))
    # adicionando o argumento n_jobs=-1 para permitir que o modelo utilize todos os processadores de cores disponíveis.
    ensemble_model = VotingClassifier(estimators=model_list, voting='hard', n_jobs=-1) 
    ensemble_model.fit(X_train, y_train)
    return ensemble_model

# função responsável por salvar modelos para uso posterior
def salvar_modelo(modelo, path):
    """Salva um modelo em um arquivo no caminho especificado.
    Args:
        model: modelo a ser salvo.
        path: caminho para onde o modelo será salvo."""
    with open(path, "wb") as f:
        pickle.dump(modelo, f)

# função responsável por carregar modelos salvos
def carregar_modelo(path):
    """Carrega um modelo salvo em um arquivo no caminho especificado.
    
    Args:
        path: caminho para o arquivo onde o modelo está salvo.
        
    Returns:
        model: modelo carregado."""
    with open(path, "rb") as f:
        model = pickle.load(f)
    return model

# função calcula as 4 principais métricas para problemas de classificação multilabel e retorna um dataframe com os valores de cada uma
def metricas_finais(X_test_test, y_test_test, modelos):
    # Lista de métricas
    metrics = ['accuracy', 'precision', 'recall', 'f1_score']
    # Dicionário para armazenar os resultados
    results = {}
    # Itera sobre todos os modelos
    for model in modelos:
        # Instancia o modelo
        model_obj = model
        # Realiza a previsão
        y_pred = model_obj.predict(X_test_test)
        # Calcula as métricas
        accuracy = accuracy_score(y_test_test, y_pred)
        precision = precision_score(y_test_test, y_pred, average='macro')
        recall = recall_score(y_test_test, y_pred, average='macro')
        f1 = f1_score(y_test_test, y_pred, average='macro')
        # Adiciona os resultados ao dicionário
        results[str(model).split('(')[0]] = [accuracy, precision, recall, f1]
    # Cria o DataFrame com os resultados
    df = pd.DataFrame(results, index=metrics).T
    return df

### Coletando os dados

In [ ]:
# Esse código importa dados de um arquivo CSV, seleciona algumas colunas, remove linhas com dados faltantes, 
# substitui valores faltantes, reseta os índices e amostra 5 linhas aleatórias.

df_coment = processador.importar_dados('/kaggle/input/brazilian-ecommerce/olist_order_reviews_dataset.csv')
df_coment = df_coment[['order_id','review_id','review_comment_message','review_comment_title','review_score']]
df_coment.dropna(subset=['review_comment_message'], inplace=True)
df_coment['review_comment_title'] = df_coment['review_comment_title'].replace(np.nan, 'Não Respondeu')
df_coment.reset_index(inplace=True)
df_coment.drop('index',axis=1,inplace=True)
df_coment.loc[df_coment['review_score'] == 2,'review_score'] = 1
df_coment.loc[df_coment['review_score'] == 4,'review_score'] = 5
df_coment.sample(5)

In [ ]:
# Esse código importa dados, seleciona algumas colunas, e amostra 5 linhas aleatórias. 

df_item_do_pedido = processador.importar_dados('/kaggle/input/brazilian-ecommerce/olist_order_items_dataset.csv')
df_item_do_pedido = df_item_do_pedido[['order_id','product_id','seller_id','price','freight_value']]
df_item_do_pedido.sample(5)

In [ ]:
# Esse código importa dados, seleciona algumas colunas, e amostra 5 linhas aleatórias.

df_vendas = processador.importar_dados('/kaggle/input/brazilian-ecommerce/olist_orders_dataset.csv')
df_vendas = df_vendas[['order_id','customer_id','order_status','order_approved_at','order_delivered_customer_date','order_estimated_delivery_date']]
df_vendas.sample(5)

### Pré-processamento de dados

#### Lista das etapas utilizadas:

1. Limpeza de dados: É necessário remover caracteres especiais, pontuação, espaços em branco e outros caracteres indesejados dos dados para que possamos obter resultados mais precisos. Se não limparmos os dados, os resultados podem ser imprecisos.

2. Tokenização: É necessário separar os dados em tokens usando a biblioteca NLTK para que possamos trabalhar com eles. Se não tokenizarmos os dados, não poderemos realizar nenhuma análise.

3. Stemming: É necessário aplicar o processo de stemming nos tokens usando a biblioteca NLTK para reduzir a dimensionalidade dos dados. Se não aplicarmos o processo de stemming, os dados podem ter uma dimensionalidade muito alta, aumento diretamente o custo computacional.

4. Lemmatization: É necessário aplicar o processo de lematização nos tokens usando a biblioteca NLTK para reduzir a dimensionalidade dos dados.

5. Remoção de stopwords: É necessário remover as stopwords para reduzir a dimensionalidade dos dados.

6. Vectorização: É necessário transformar os tokens em vetores para que possamos trabalhar com eles. Se não vectorizarmos os dados, não poderemos realizar nenhuma análise.

7. Normalização: É necessário normalizar os vetores usando a biblioteca Scikit-learn para que possamos obter resultados mais precisos. Se não normalizarmos os dados, os resultados podem ser imprecisos.

In [ ]:
# Esse código cria um conjunto de dados X e y para treinamento de um modelo de Machine Learning a partir de três 
# dataframes diferentes (df_coment, df_vendas e df_item_do_pedido). O conjunto de dados X é obtido pela função compactar(), 
# que seleciona df_coment e outros dois dataframes df_vendas e df_item_do_pedido. 
# O conjunto de dados y é obtido pela variável "target" (review_score). A variável "key ou chave primária" (order_id) é usada para unir os três dataframes.

dfs = [df_coment.sample(1000), df_vendas,df_item_do_pedido]
target='review_score'
key='order_id'
X, y = compactar(dfs, target, key)
X

In [ ]:
# Esse código cria um dicionário de tratamentos, executa a limpeza e tratamento dos dados, e depois faz a divisão entre treino e teste.

tratamentos = {
            'remove_acentuacao': True, 
            'texto_e_numero': True, 
            'remove_carac_dif': True, 
            'tokens': True, 
            'stemming_radical': True, 
            'stopwords': True
            }
X_full_norm, X_coments, X_items = limpa_e_trata(X, tratamentos)
X_train, X_test, y_train, y_test = train_test_split(X_full_norm, y, test_size=0.3, stratify=y, shuffle=True, random_state=1)
X_train.shape, X_test.shape, y_train.shape, y_test.shape 

In [ ]:
display(X_train.head(3))
X_train.shape

In [ ]:
# Esse código reduz a dimensionalidade dos dados de treinamento (X_train) para 10, 25, 50 e 150 características usando um algoritmo de redução de dimensionalidade.

X_train_reduzido10 = reducao_dimensionalidade(X_train, y_train, n_features=10)
X_train_reduzido25 = reducao_dimensionalidade(X_train, y_train, n_features=25)
X_train_reduzido50 = reducao_dimensionalidade(X_train, y_train, n_features=50)
X_train_reduzido150 = reducao_dimensionalidade(X_train, y_train, n_features=150)

In [ ]:
#Esse código executa vários modelos de classificação com diferentes datasets, calculando uma métrica de acurácia, para comparar os melhores resultados.

datas = [X_train_reduzido10,X_train_reduzido25,X_train_reduzido50,X_train_reduzido150]
modelos = [LGBMClassifier, GradientBoostingClassifier, LogisticRegression, SVC]
metrica = 'acuracia'
df_metricas_geral = pd.DataFrame(columns=['modelo',metrica,'n_colunas'])
n=1
for dataframe in datas:
    df_metricas, fig_boxplot = melhores_modelos(dataframe, y_train, modelos=modelos)
    fig_boxplot.show()
    df_metricas_geral = pd.concat([df_metricas_geral, df_metricas])
    print(f'{n}º (de {len(datas)}) Dataset Concluido!. \ncols: {dataframe.shape[1]}')
    n += 1

In [ ]:
# Este código salva um dataframe como um arquivo CSV, agrupa os dados por dois campos, e os ordena por acurácia.

df_metricas_geral.to_csv('df_metricas_geral_cv_nFull.csv',index=False)
#df_metricas_geral = pd.read_csv('/kaggle/working/df_metricas_geral_cv.csv')
df_metricas_geral = df_metricas_geral.groupby(['modelo','n_colunas']).agg('mean').reset_index()
df_metricas_geral.sort_values('acuracia',ascending=False)

In [ ]:
# Esse código desenha um gráfico de linha usando os dados de um DataFrame, plotando a acurácia em relação ao número de colunas para cada modelo.
px.line(df_metricas_geral, x='n_colunas',y='acuracia', color='modelo')

In [ ]:
display(X_test.head(3))
X_test.shape

In [ ]:
# Esse código cria um novo conjunto de dados (dataset_campeão) a partir da variável X_train_reduzido150 e atribui as 
# variáveis X_train e X_test a esses dados. Ele também exibe o cabeçalho de 3 linhas do X_test e exibe o formato do X_test.

dataset_campeao = X_train_reduzido150.copy()
X_train = dataset_campeao
X_test = X_test[dataset_campeao.columns.to_list()]
display(X_test.head(3))
X_test.shape

### Tunagem dos melhores modelos

#### métricas avaliadas
• accuracy: essa métrica mede quantas vezes o modelo acertou as previsões. Quanto mais perto de 100%, melhor o modelo está se saindo.

• precision: essa métrica mede o quanto o modelo consegue acertar quando diz que algo é da classe X. Se o modelo tem uma precisão alta, é porque ele raramente diz que algo é da classe X quando na verdade não é.

• recall: essa métrica mede o quanto o modelo consegue encontrar todas as coisas da classe X. Se o modelo tem um recall alto, é porque ele consegue encontrar a maioria das coisas da classe X.

• f1-score*: essa métrica combina a precisão e o recall em um único número. Quanto mais perto de 100%, melhor o modelo está se saindo.


In [ ]:
# Esse código cria um modelo de Gradient Boosting e realiza uma otimização de parâmetros para encontrar os melhores resultados.
params_gbm = {
    "loss": ["deviance", "exponential"], # função de custo para minimizar
    "learning_rate": scipy.stats.uniform(0.01, 0.2), # taxa de aprendizado
    "n_estimators": scipy.stats.randint(50, 500), # número de árvores
    "max_depth": scipy.stats.randint(1, 10), # profundidade máxima das árvores
    "min_samples_split": scipy.stats.randint(2, 20), # número mínimo de amostras para realizar split em um nó
    "min_samples_leaf": scipy.stats.randint(1, 20), # número mínimo de amostras em cada folha das árvores
    "max_features": ["auto", "sqrt", "log2", None], # número máximo de features a serem consideradas em cada split
}

modelo1_gb = GradientBoostingClassifier(random_state=42)
modelo1_gb, best_params_gb, accuracy_gb, df_report_gb, mat_conf_gb = tunagem_de_modelo(modelo1_gb, X_train, y_train, params_gbm)

In [ ]:
# Mostrando os melhores parametros
best_params_gb

In [ ]:
# Mostrando a acurácia do modelo nos dados de treino
accuracy_gb

In [ ]:
# Mostrando as principais métricas do modelo nos dados de treino
df_report_gb

In [ ]:
# Mostrando a matriz de confusão usando dados de treino
mat_conf_gb

In [ ]:
# Criando um dataframe com dados sobre os comentários negativos (classe 1)

models_metricas_neg = df_report_gb[['1.0']]
models_metricas_neg.columns = ['mod1_gb']
models_metricas_neg

In [ ]:
# Salvando e Carregando o modelo

#salvar_modelo(modelo1_gb,'/kaggle/working/modelo1_gb_nFull.pkl')
#modelo1_gb = carregar_modelo('/kaggle/input/modelo1-gb/modelo1_gb.pkl')

---

In [ ]:
# Esse código cria um modelo de LGBMClassifier e realiza uma otimização de parâmetros para encontrar os melhores resultados.
params_lgbm = {
    "learning_rate": scipy.stats.uniform(0.01, 0.2), # taxa de aprendizado
    "n_estimators": scipy.stats.randint(50, 500), # número de árvores
    "max_depth": scipy.stats.randint(1, 10), # profundidade máxima das árvores
    "num_leaves": scipy.stats.randint(2, 50), # número máximo de folhas em cada árvore
    "min_child_samples": scipy.stats.randint(1, 20), # número mínimo de amostras em cada nó interno da árvore
    "subsample": scipy.stats.uniform(0.01, 1.0), # proporção da amostra a ser usada para treinar cada árvore
    "subsample_freq": scipy.stats.randint(1, 10), # frequência com que o bagging é aplicado
    "colsample_bytree": scipy.stats.uniform(0.01, 1.0), # proporção de features a serem consideradas em cada split
}

modelo2_lgbm = LGBMClassifier(random_state=123)
modelo2_lgbm, best_params_lgbm, accuracy_lgbm, df_report_lgbm, mat_conf_lgbm = tunagem_de_modelo(modelo2_lgbm, X_train, y_train, params_lgbm)

In [ ]:
# Mostrando os melhores parametros
best_params_lgbm

In [ ]:
# Mostrando a acurácia do modelo nos dados de treino
accuracy_lgbm

In [ ]:
# Mostrando as principais métricas do modelo nos dados de treino
df_report_lgbm

In [ ]:
# Mostrando a matriz de confusão usando dados de treino
mat_conf_lgbm

In [ ]:
# Criando um dataframe com dados sobre os comentários negativos (classe 1)

models_metricas_neg['mod2_lgbm'] = df_report_lgbm[['1.0']]
models_metricas_neg

In [ ]:
# Salvando e Carregando o modelo

#salvar_modelo(modelo2_lgbm,'/kaggle/working/modelo2_lgbm_nFull.pkl')
#modelo2_lgbm = carregar_modelo('/kaggle/input/modelo2-lgbm/modelo2_lgbm.pkl')

---

In [ ]:
# Esse código cria um modelo de LogisticRegression e realiza uma otimização de parâmetros para encontrar os melhores resultados.
params_logistic_reg = {
    "penalty": ["l1", "l2"], # tipo de regularização a ser utilizada
    "C": scipy.stats.uniform(0.01, 10), # valor do inverso do parâmetro de regularização
    "solver": ["newton-cg", "lbfgs", "liblinear", "sag", "saga"], # solver a ser utilizado para otimizar a função de custo
    "max_iter": scipy.stats.randint(100, 1000), # número máximo de iterações do solver
}

modelo3_lr = LogisticRegression()
modelo3_lr, best_params_lr, accuracy_lr, df_report_lr, mat_conf_lr = tunagem_de_modelo(modelo3_lr, X_train, y_train, params_logistic_reg)

In [ ]:
# Mostrando os melhores parametros
best_params_lr

In [ ]:
# Mostrando a acurácia do modelo nos dados de treino
accuracy_lr

In [ ]:
# Mostrando as principais métricas do modelo nos dados de treino
df_report_lr

In [ ]:
# Mostrando a matriz de confusão usando dados de treino
mat_conf_lr

In [ ]:
# Criando um dataframe com dados sobre os comentários negativos (classe 1)

models_metricas_neg['mod3_reg_logist'] = df_report_lr['1.0']
models_metricas_neg

In [ ]:
# Salvando e Carregando o modelo

#salvar_modelo(modelo3_lr,'/kaggle/working/modelo3_lr_nFull.pkl')
#modelo3_lr = carregar_modelo('/kaggle/input/modelo3_lr/modelo3_lr.pkl')

---

In [ ]:
# Esse código cria um modelo de SVC e realiza uma otimização de parâmetros para encontrar os melhores resultados.
params_svm = {
    "kernel": ["linear", "poly", "rbf", "sigmoid"],
    "C": scipy.stats.uniform(0.01, 10),
    "degree": scipy.stats.randint(2, 5),
    "gamma": ["scale", "auto"] + [scipy.stats.uniform(0.01, 10).rvs()]
}

# Instância do modelo
modelo4_svm = SVC(random_state=42)

# Chamada da função de tunagem de modelo
modelo4_svm, best_params_svm, accuracy_svm, df_report_svm, mat_conf_svm = tunagem_de_modelo(modelo4_svm, X_train, y_train, params_svm)

In [ ]:
# Mostrando os melhores parametros
best_params_svm

In [ ]:
# Mostrando a acurácia do modelo nos dados de treino
accuracy_svm

In [ ]:
# Mostrando as principais métricas do modelo nos dados de treino
df_report_svm

In [ ]:
# Mostrando a matriz de confusão usando dados de treino
mat_conf_svm

In [ ]:
# Criando um dataframe com dados sobre os comentários negativos (classe 1)

models_metricas_neg['mod4_SVM'] = df_report_svm['1.0']
models_metricas_neg

In [ ]:
# Salvando e Carregando o modelo

#salvar_modelo(modelo4_svm,'/kaggle/working/modelo4_svm_nFull.pkl')
#modelo4_svm = carregar_modelo('/kaggle/input/modelo4_svm/modelo4_svm.pkl')

---

In [ ]:
modelos_tunados = [modelo1_gb,modelo2_lgbm,modelo3_lr,modelo4_svm]
X_test_train, X_test_test, y_test_train, y_test_test = train_test_split(X_test, y_test, test_size=0.3, shuffle=True, random_state=42)

ensemble_model = ensemble_modelos(modelos_tunados, X_test_train, y_test_train)

In [ ]:
# Avaliando o desempenho do modelo Ensemble
y_test_pred = ensemble_model.predict(X_test_test)
df_report_ensemb = pd.DataFrame(classification_report(y_test_test, y_test_pred, output_dict=True))
conf_matrix_ensemb = confusion_matrix(y_test_test, y_test_pred)
df_conf_matrix_ensemb = pd.DataFrame(conf_matrix_ensemb, index=['Real 1', 'Real 2', 'Real 3'], columns=['Predito 1', 'Predito 2', 'Predito 3'])

In [ ]:
# Mostrando as 5 primeiras previsões
y_test_pred[0:5]

In [ ]:
# Mostrando as principais métricas do modelo nos dados de treino
df_report_ensemb

In [ ]:
# Mostrando a matriz de confusão usando dados de treino
df_conf_matrix_ensemb

In [ ]:
# Criando um dataframe com dados sobre os comentários negativos (classe 1)

models_metricas_neg['mod5_ensemble'] = df_report_ensemb['1.0']
models_metricas_neg

In [ ]:
# Baixando as metricas dos modelos para um arquivo csv
#models_metricas_neg.to_csv('df_metricas_neg_nFull.csv',index=False)

In [ ]:
# Salvando e Carregando o modelo

#salvar_modelo(ensemble_model,'/kaggle/working/ensemble_model_nFull.pkl')
#ensemble_model = carregar_modelo('/kaggle/input/ensemble-model-nfull/ensemble_model_nFull.pkl')


### Avaliação do desempenho do modelo

In [ ]:
# Esse código seleciona os modelos criados e calcula as principais métricas de avaliação, dessa vez, para dados de teste

modelos_pra_teste = [modelo1_gb,modelo2_lgbm,modelo3_lr,modelo4_svm,ensemble_model]
df_metricas_teste = metricas_finais(X_test, y_test, modelos_pra_teste)
df_metricas_teste.style.background_gradient()

In [ ]:
#df_metricas_teste.to_csv('df_metricas_dados_de_teste_nFull.csv',index=False)

O modelo VotingClassifier obteve as melhores métricas de acurácia e f1-score. Ele é um modelo ensemble, o que significa que é composto por múltiplos modelos trabalhando juntos, o que pode resultar em uma previsão mais robusta e precisa. O modelo GradientBoostingClassifier foi o segundo colocado e é mais rápido que o VotingClassifier, mas possui métricas um pouco inferiores. Ele pode ser uma opção em casos onde o tempo é um fator crítico.

In [ ]:
# Esse código cria um gráfico de barras comparando as métricas de desempenho de quatro modelos de classificação.
import plotly.graph_objects as go

x = ['GradientBoostingClassifier', 'LGBMClassifier', 'SVC', 'VotingClassifier']

# Criando os subplots
fig = go.Figure()

# Subplot 1
fig.add_trace(go.Bar(x=df_metricas_teste.index, y=df_metricas_teste['accuracy'], name='Acuracia'))

# Subplot 2
fig.add_trace(go.Bar(x=df_metricas_teste.index, y=df_metricas_teste['precision'], name='Precisão'))

# Subplot 3
fig.add_trace(go.Bar(x=df_metricas_teste.index, y=df_metricas_teste['recall'], name='Recall'))

# Subplot 4
fig.add_trace(go.Bar(x=df_metricas_teste.index, y=df_metricas_teste['f1_score'], name='F1 Score'))

# Update layout
fig.update_layout(title='Métricas do Desempenho dos modelos')

fig.show()

### Apresentação dos resultados, Considerações e Upgrades

Após analisarmos os comentários de reviews de e-commerce com nosso projeto de classificação de sentimentos, pudemos constatar que o modelo VotingClassifier foi robusto e preciso para a solução do problema. Além disso, os resultados obtidos pelo modelo GradientBoostingClassifier foram promissores, oferecendo uma alternativa viável quando o tempo de processamento é uma questão crucial.

Embora os resultados obtidos neste projeto sejam promissores, é importante lembrar que eles foram obtidos a partir de um conjunto específico de dados e parâmetros. Portanto, é recomendado realizar testes adicionais para avaliar a generalização dos modelos em outros contextos e garantir sua eficiência. Além disso, é sempre importante considerar se outros modelos ou técnicas poderiam ser úteis no problema em questão. Como profissionais da área de ciência de dados, é fundamental sermos críticos e buscar sempre a melhor solução para cada problema, de forma a garantir resultados precisos e confiáveis. Isso nos permite trabalhar de forma eficiente e proporcionar soluções valiosas para nossos clientes.

<a href="mailto:ueliton_viana@outlook.com"><i style="font-size:25px; margin-right:10px" class="fa fa-envelope"></i></a>
<a href="https://wa.me/5549984233608"><i style="font-size:25px; margin-right:10px" class="fa fa-whatsapp"></i></a>
<a href="https://www.linkedin.com/in/ueliton-viana/"><i style="font-size:25px; margin-right:10px" class="fa fa-linkedin"></i></a>
<a href="https://github.com/Uellwar"><i style="font-size:25px; margin-right:10px" class="fa fa-github"></i></a>
<a href="https://www.instagram.com/uelitonviana_/"><i style="font-size:25x; margin-right:10px" class="fa fa-instagram"></i></a>
<a href="https://www.youtube.com/channel/UCX-MsX0Tt-iGBScRhsDfSJA"><i style="font-size:25px; margin-right:10px" class="fa fa-youtube"></i></a>
